In [ ]:
%pip install peft
%pip install -i https://pypi.org/simple/ bitsandbytes
%pip install bigjson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import re

# Sample input text
# Regular expressions to capture both "Unknown" and specific values
patterns = {
    'Abeta': r'Abeta: (Unknown.|This patient abeta burden is ([\w\s]+).)',
    'Meta tau': r'Meta tau: (Unknown.|This patient Meta tau burden is ([\w\s]+).)',
    'MMSE rate': r'MMSE rate: (Unknown.|The MMSE is ([\w\s]+).)',
    'Treatment': r'Treatment: (Unknown.|This participant is ([\w\s]+) to be a candidate for anti-amyloid treatment.)'
}

# Function to extract the full sentence
def extract_answers(text, patterns):
    results = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            # Return the full sentence as the answer
            results[key] = match.group(1)
        else:
            results[key] = "Unknown"
    return results


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 119.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
peft_model_id="/content/drive/MyDrive/LLaMA-Factory/saves/LLaMA3-8B/lora/train_2024-AD-Dec-27"
model=AutoModelForCausalLM.from_pretrained(peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# peft_model_id="/content/drive/MyDrive/LLaMA-Factory/saves/LLaMA3-8B/lora/train_2025-AD_Jan_3_zero/checkpoint-1"
# model=AutoModelForCausalLM.from_pretrained(peft_model_id)
# tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



In [ ]:
instruction_llm="You are a specialist in diagnosing and forecasting the disease progression for Alzheimer's disease participants. Your task is to predict the following for a patient based on their medical records: the current amyloid beta (abeta) burden in the brain PET scan, the current tau burden of the temporal meta regions of interest (ROIs) in the brain PET scan, Mini-Mental State Examination (MMSE) score change rate, and whether this participant is the ideal candidate for anti-amyloid treatment. Please provide the prediction in this structured format: Abeta: Unknown or This patient abeta burden is [Negative/Intermediate negative/Weakly positive/Clearly positive]. Meta tau: Unknown or This patient Meta tau burden is [Very low/Low/High/Super high]. MMSE rate: Unknown or The MMSE is [Slow decline/Moderate decline/Fast decline/Stable]. Treatment: Unknown or This participant is [ideal/unlikely] to be a candidate for anti-amyloid treatment."

In [ ]:
tokenizer.encode("Abeta: This patient abeta burden is Negative. Meta tau: This patient Meta tau burden is High. MMSE rate: The MMSE is Stable. Treatment: This participant is ideal to be a candidate for anti-amyloid treatment.")

[128000,
 5953,
 1955,
 25,
 1115,
 8893,
 671,
 1955,
 23104,
 374,
 51957,
 13,
 16197,
 32923,
 25,
 1115,
 8893,
 16197,
 32923,
 23104,
 374,
 5234,
 13,
 22403,
 937,
 4478,
 25,
 578,
 22403,
 937,
 374,
 84441,
 13,
 31969,
 25,
 1115,
 25923,
 374,
 10728,
 311,
 387,
 264,
 9322,
 369,
 7294,
 12,
 27322,
 52196,
 6514,
 13]

In [ ]:
tokenizer.decode(84441)

' Stable'

In [ ]:
def calculate_probability_new(outputs, generated_tokens,i,tail=""):
  scores = outputs.scores
  from torch.nn.functional import softmax
  top_k = 20  # Define the number of top tokens to consider
  probabilities = [softmax(score, dim=-1) for score in scores]
  generated_tokens = tokenizer.convert_ids_to_tokens(generated_tokens[0])
  import torch
  top_k_tokens_and_probs = []
  for position_probs in probabilities:
    top_k_probs, top_k_indices = torch.topk(position_probs[0], top_k)
    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())
    top_k_tokens_and_probs.append(list(zip(top_k_tokens, top_k_probs.tolist()))) # all top 10 tokens in each generated positions
  expected_tokens=["ĠNegative", "ĠIntermediate", "ĠWeak", "ĠClearly", "ĠUnknown", "ĠSlow", "ĠModerate", "ĠFast", "ĠStable", "ĠVery", "ĠLow", "ĠHigh", "ĠSuper","Ġideal","Ġunlikely"]
  # if abeta_answer=="Unknown" and tau_answer=="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer!="Unknown" and tau_answer=="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating","ĠClearly", "ĠIntermediate", "ĠWeak", "ĠNegative","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer=="Unknown" and tau_answer!="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating", "ĠIntermediate","ĠLow","ĠVery","ĠHigh","ĠSuper","Ġincreasing","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer!="Unknown" and tau_answer!="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating","ĠClearly", "ĠIntermediate", "ĠWeak", "ĠNegative", "ĠLow","ĠHigh","ĠSuper","Ġincreasing","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  expected_token_ids = [tokenizer.convert_tokens_to_ids(token) for token in expected_tokens]

  token_probabilities = {token: [] for token in expected_tokens}

  for idx, token_prob_list in enumerate(top_k_tokens_and_probs):
    for token, prob in token_prob_list:
        if token in expected_tokens:
            token_probabilities[token].append((idx, prob))
  import json
  # Specify the file path
  # file_path = f'/content/drive/MyDrive/AD_Jan_3_Zero/ADNI/probability/output_{i}'+tail+'.json'
  # print(token_probabilities)
  # with open(file_path, 'w') as json_file:
  #   json.dump(token_probabilities, json_file, indent=4)
  return 0

In [ ]:
## ADNI inference

In [ ]:
import pandas as pd
adni_index=pd.read_csv("/content/drive/MyDrive/AD_Dec_27/data/Incomplete/index_complete.csv")

In [ ]:
adni_list=list(adni_index["Index"])

In [ ]:
inputs = tokenizer(
            [
                alpaca_prompt.format(
                    "You are a specialist in diagnosing and forecasting the disease progression for Alzheimer's disease participants. Your task is to predict the following for a patient based on their medical records: the current amyloid beta (abeta) burden in the brain PET scan, the current tau burden of the temporal meta regions of interest (ROIs) in the brain PET scan, Mini-Mental State Examination (MMSE) score change rate, and whether this participant is the ideal candidate for anti-amyloid treatment. Please provide the prediction in this structured format: Abeta: Unknown or This patient abeta burden is [Negative/Intermediate negative/Weakly positive/Clearly positive]. Meta tau: Unknown or This patient Meta tau burden is [Very low/Low/High/Very high]. MMSE rate: Unknown or The MMSE is [Slow decline/Moderate decline/Fast decline/Stable]. Provide your reasons for prediction of Abeta, Meta tau, and MMSE.", # instruction
                    "Patient Baseline information:\n Demographic information:\n  This patient is born in 1943, Right-handed, Retired, Male,  and the marital status is Married,  with 16 years of education, a self-identified race White, ethnicity Not Hispanic or Latino. \n Genetic information:\n This is patient is 1 APOE4 allele carrier, and is not APOE2 allele carrier. \n Family history:\n Mother: patient's mother has dementia. patient's mother has Alzheimer's disease.  The mother's symptoms start at approximate 73.0 years old. Father: patient's father does not have dementia.  Sibling: This patient has 2 siblings, 2 has no dementia, 0 has dementia. This visit is the 1 time visit. In total, the patient has 1 visits on 2019-09-04. The trajectory of features during these visits (from the very first to the latest) are:\n \n Vitals:\n Age: 75.9, Weight (in kg): 85.0, Height (in cm): 178.0, Systolic blood pressure (mmHg): 120, Diastolic blood pressure (mmHg): 60, Seated Pulse Rate (per minute): 76, Respirations (per minute): 18, Temperature (in Celsius): 36.2, \n Medication:\n Medication: Aricept | Anti-depressant medication, \n Cognitive test:\n Mini-Mental State Examination score: 30, Geriatric Depression Scale: 0, Clinical Dementia Rating Scale Sum of Boxes Scores: 2.5, Alzheimer's Disease Assessment Scale (11 items): 8.0, Alzheimer's Disease Assessment Scale (13 items): 13.0, Question 4 score in the 11-item test: 5, Rey Auditory Verbal Learning Test (Immediate): 37, Rey Auditory Verbal Learning Test (Learning): 4, Rey Auditory Verbal Learning Test (Forgetting): 6, Rey Auditory Verbal Learning Test (Percent forgetting): 60.0, Delayed Total Recall test: 5, Trail Making Test Part B Time: 63, Functional Assessment Questionnaire: 6, Montreal Cognitive Assessment: 25, PACC score: -3.88, Everyday Cognition test (self-reported Memory): 3.6, Everyday Cognition test (self-reported Language): 3.0, Everyday Cognition test (self-reported Visuospatial Abilities): 2.7, Everyday Cognition test (self-reported Planning): 3.8, Everyday Cognition test (partner-reported Organization): 3.0, Everyday Cognition test (partner-reported Divided attention): 3.5, Everyday Cognition test (partner-reported Total scores): 2.4, \n AD biomarker in Cerebrospinal Fluid:\n Amyloid beta 40 in Cerebrospinal Fluid (pg/mL): 19990.0, Amyloid beta 42 in Cerebrospinal Fluid (pg/mL): 590.8, Tau protein concentration in Cerebrospinal Fluid (pg/mL): 308.3, Phosphorylated tau 181 protein in Cerebrospinal Fluid (pg/mL): 31.72, \n Fluorodeoxyglucose PET image:\n average FDG-PET of angular, temporal, and posterior cingulate: 1.06, \n Brain Volume in MRI:\n Left Hippocampus: 4121, Right Hippocampus: 4306, Left Lateral Ventricle: 24690, Right Lateral Ventricle: 29921, Total intracranial: 1644260, \n Brain Thickness in MRI:\n Left Entorhinal (10E-3): 3264, Right Entorhinal (10E-3): 3157, Left Fusiform (10E-3): 2655, Right Fusiform (10E-3): 2767, Left Inferior Parietal (10E-3): 2296, Right Inferior Parietal (10E-3): 2302, Left Inferior Temporal (10E-3): 2683, Right Inferior Temporal (10E-3): 2716, Left Middle Temporal (10E-3): 2793, Right Middle Temporal (10E-3): 2768, Left Precuneus (10E-3): 2314, Right Precuneus (10E-3): 2491, Left Insula (10E-3): 2913, Right Insula (10E-3): 2866, Left Meta ROI (10E-3): 2703, Right Meta ROI (10E-3): 2724, \n Diagnosis:\n Diagnosis: Mildly cognitive impairment, \n Other health condition:\n Diagnosis of Depression: No depressive symptoms, Diagnosis of Parkinson's disease: No parkinsonism symptoms.", # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
outputs =model.generate(**inputs, max_new_tokens = 1000, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True)
transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
print(tokenizer.decode(generated_tokens.cpu()[0]))

 patient abeta burden is Clearly positive. The patient is 1 APOE4 allele carrier, which is a risk factor for Alzheimer's disease. The patient's amyloid beta 40 in Cerebrospinal Fluid is 19990.0 pg/mL, and amyloid beta 42 in Cerebrospinal Fluid is 590.8 pg/mL, indicating a high amyloid burden. The patient's average FDG-PET of angular, temporal, and posterior cingulate is 1.06, which is low, indicating reduced glucose metabolism in these regions.

Meta tau: This patient Meta tau burden is Very high. The patient's tau protein concentration in Cerebrospinal Fluid is 308.3 pg/mL, which is high, indicating a high tau burden. The patient's phosphorylated tau 181 protein in Cerebrospinal Fluid is 31.72 pg/mL, which is also high, indicating a high level of tau pathology.

MMSE rate: The MMSE is Fast decline. The patient's Mini-Mental State Examination score is 30, which is low, indicating significant cognitive decline. The patient's Alzheimer's Disease Assessment Scale (11 items) is 8.0, and Al

In [ ]:
import numpy as np
import bigjson
import json
import torch
import torch.nn.functional as F
import re
decrease_answer=[]
not_answer=[]
decrease_prob=[]
not_prob=[]
json_filepath="/content/drive/MyDrive/AD_Jan_3_Zero/ADNI/adni_generation_zero.json"

data_for_json=[]
with open('/content/drive/MyDrive/AD_Dec_27/data/adni_test_Dec27.json', 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
      # if i in adni_list:
        dict_data = data.to_python()
        values = extract_answers(dict_data["output"], patterns)
        abeta_status=values["Abeta"]
        meta_status=values["Meta tau"]
        mmse_rate=values["MMSE rate"]
        treatment_status=values["Treatment"]
        if abeta_status=="Unknown." and meta_status=="Unknown." and mmse_rate!="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: Unknown. Meta tau: Unknown. MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output": "Abeta: Unknown. Meta tau: Unknown. MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }
            calculate_probability_new(outputs,generated_tokens,i,"other_unknown_mmse")
        if abeta_status!="Unknown." and meta_status=="Unknown." and mmse_rate=="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                   instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens,i,"other_unknown_abeta")
            entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output": "Abeta: This "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }

        if abeta_status!="Unknown." and meta_status=="Unknown." and mmse_rate!="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens =30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens,i,"abeta_known_tau_unknown_mmse_known_abeta")

            indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_abeta = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+ decoded_text_abeta+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_known_tau_unknown_mmse_known_mmse")
              entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output":  "Abeta: This "+ decoded_text_abeta+" MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }

        if abeta_status=="Unknown." and meta_status!="Unknown." :

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: Unknown. Meta tau: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
        #
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True, return_dict_in_generate=True,do_sample=False,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens, i,"abeta_unknown_tau_known_meta_tau")

            indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_new_meta_tau = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: Unknown. Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_unknown_tau_known_mmse")
              entry = {
                    "instruction":  instruction_llm,
                    "input":dict_data["input"],
                    "output": "Abeta: Unknown. Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                    "index":i
                }
        if abeta_status!="Unknown." and meta_status!="Unknown." :

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
        #
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True, return_dict_in_generate=True,do_sample=False,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens, i,"abeta_tau_known_abeta")

            indices = (generated_tokens.cpu()[0] == tokenizer(".Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_new_tau = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+decoded_text_new_tau+" Meta tau: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_meta_tau")
              indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices) > 0:
                first_occurrence = indices[0]
                row= first_occurrence.item()
                values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
                decoded_text_new_meta_tau = tokenizer.decode(values_before_8)
                inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
                outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
                transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
                input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
                generated_tokens = outputs.sequences[:, input_length:]
                calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_mmse")
                indices = (generated_tokens.cpu()[0] == tokenizer(".Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer("Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer(" Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer(".T")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices) > 0:
                  first_occurrence = indices[0]
                  row= first_occurrence.item()
                  values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
                  decoded_text_mmse = tokenizer.decode(values_before_8)
                  inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+decoded_text_mmse+" Treatment: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
                  outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
                  transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
                  input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
                  generated_tokens = outputs.sequences[:, input_length:]
                  calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_treatment")
                  entry = {
                    "instruction":  instruction_llm,
                    "input":dict_data["input"],
                    "output": "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+decoded_text_mmse+" Treatment: This "+tokenizer.decode(generated_tokens.cpu()[0]),
                    "index":i
                }

        print(entry["output"])
        data_for_json.append(entry)
        with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
           json.dump(data_for_json, jsonfile, indent=4, ensure_ascii=False)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


流式输出内容被截断，只能显示最后 5000 行内容。
The patient
{'ĠNegative': [], 'ĠIntermediate': [], 'ĠWeak': [], 'ĠClearly': [], 'ĠUnknown': [(3, 0.004693683236837387), (6, 0.00027505794423632324), (7, 0.5154842734336853), (26, 0.014013992622494698)], 'ĠSlow': [(3, 0.16806361079216003)], 'ĠModerate': [(3, 0.13088808953762054)], 'ĠFast': [(3, 0.00454927422106266)], 'ĠStable': [(3, 0.28588584065437317)], 'ĠVery': [], 'ĠLow': [], 'ĠHigh': [], 'ĠSuper': [], 'Ġideal': [], 'Ġunlikely': []}
Abeta: Unknown. Meta tau: Unknown. MMSE rate: The  MMSE is Stable. Treatment: Unknown. 

### Reasoning:
The patient's amyloid beta burden and meta tau burden are unknown because the necessary
{'ĠNegative': [], 'ĠIntermediate': [], 'ĠWeak': [], 'ĠClearly': [], 'ĠUnknown': [(7, 0.000619950529653579), (8, 0.5137041807174683), (27, 0.011507401242852211)], 'ĠSlow': [(3, 0.17488905787467957)], 'ĠModerate': [(3, 0.13620373606681824)], 'ĠFast': [(3, 0.008913698606193066)], 'ĠStable': [(3, 0.17488905787467957), (5, 0.0001684135495452210

In [ ]:
def calculate_probability_new(outputs, generated_tokens,i,tail=""):
  scores = outputs.scores
  from torch.nn.functional import softmax
  top_k = 20  # Define the number of top tokens to consider
  probabilities = [softmax(score, dim=-1) for score in scores]
  generated_tokens = tokenizer.convert_ids_to_tokens(generated_tokens[0])
  import torch
  top_k_tokens_and_probs = []
  for position_probs in probabilities:
    top_k_probs, top_k_indices = torch.topk(position_probs[0], top_k)
    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())
    top_k_tokens_and_probs.append(list(zip(top_k_tokens, top_k_probs.tolist()))) # all top 10 tokens in each generated positions
  expected_tokens=["ĠNegative", "ĠIntermediate", "ĠWeak", "ĠClearly", "ĠUnknown", "ĠSlow", "ĠModerate", "ĠFast", "ĠStable", "ĠVery", "ĠLow", "ĠHigh", "ĠSuper","Ġideal","Ġunlikely"]
  # if abeta_answer=="Unknown" and tau_answer=="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer!="Unknown" and tau_answer=="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating","ĠClearly", "ĠIntermediate", "ĠWeak", "ĠNegative","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer=="Unknown" and tau_answer!="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating", "ĠIntermediate","ĠLow","ĠVery","ĠHigh","ĠSuper","Ġincreasing","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer!="Unknown" and tau_answer!="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating","ĠClearly", "ĠIntermediate", "ĠWeak", "ĠNegative", "ĠLow","ĠHigh","ĠSuper","Ġincreasing","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  expected_token_ids = [tokenizer.convert_tokens_to_ids(token) for token in expected_tokens]

  token_probabilities = {token: [] for token in expected_tokens}

  for idx, token_prob_list in enumerate(top_k_tokens_and_probs):
    for token, prob in token_prob_list:
        if token in expected_tokens:
            token_probabilities[token].append((idx, prob))
  import json
  # Specify the file path
  file_path = f'/content/drive/MyDrive/AD_Jan_3_Zero/HABS-HD/probability/output_{i}'+tail+'.json'
  print(token_probabilities)
  with open(file_path, 'w') as json_file:
    json.dump(token_probabilities, json_file, indent=4)
  return 0

In [ ]:
# HABS-HD

In [ ]:
import pandas as pd
habs_index=pd.read_csv("/content/drive/MyDrive/AD_Dec_27/data/Incomplete/index_complete_habs.csv")
habs_list=list(habs_index["Index"])

In [ ]:
import numpy as np
import bigjson
import json
import torch
import torch.nn.functional as F
import re
decrease_answer=[]
not_answer=[]
decrease_prob=[]
not_prob=[]
json_filepath="/content/drive/MyDrive/AD_Jan_3_Zero/HABS-HD/habs_hd_generation.json"

data_for_json=[]
with open('/content/drive/MyDrive/AD_Dec_27/data/habs_test_Dec27.json', 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
      # if i in habs_list:
        dict_data = data.to_python()
        values = extract_answers(dict_data["output"], patterns)
        abeta_status=values["Abeta"]
        meta_status=values["Meta tau"]
        mmse_rate=values["MMSE rate"]
        treatment_status=values["Treatment"]
        if abeta_status=="Unknown." and meta_status=="Unknown." and mmse_rate!="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: Unknown. Meta tau: Unknown. MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output": "Abeta: Unknown. Meta tau: Unknown. MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }
            calculate_probability_new(outputs,generated_tokens,i,"other_unknown_mmse")
        if abeta_status!="Unknown." and meta_status=="Unknown." and mmse_rate=="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                   instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens,i,"other_unknown_abeta")
            entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output": "Abeta: This "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }

        if abeta_status!="Unknown." and meta_status=="Unknown." and mmse_rate!="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens =30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens,i,"abeta_known_tau_unknown_mmse_known_abeta")

            indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_abeta = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+ decoded_text_abeta+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_known_tau_unknown_mmse_known_mmse")
              entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output":  "Abeta: This "+ decoded_text_abeta+" MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }

        if abeta_status=="Unknown." and meta_status!="Unknown." :

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: Unknown. Meta tau: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
        #
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True, return_dict_in_generate=True,do_sample=False,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens, i,"abeta_unknown_tau_known_meta_tau")

            indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_new_meta_tau = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: Unknown. Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_unknown_tau_known_mmse")
              entry = {
                    "instruction":  instruction_llm,
                    "input":dict_data["input"],
                    "output": "Abeta: Unknown. Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                    "index":i
                }
        if abeta_status!="Unknown." and meta_status!="Unknown." :

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
        #
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True, return_dict_in_generate=True,do_sample=False,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens, i,"abeta_tau_known_abeta")

            indices = (generated_tokens.cpu()[0] == tokenizer(".Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_new_tau = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+decoded_text_new_tau+" Meta tau: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_meta_tau")
              indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices) > 0:
                first_occurrence = indices[0]
                row= first_occurrence.item()
                values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
                decoded_text_new_meta_tau = tokenizer.decode(values_before_8)
                inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
                outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
                transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
                input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
                generated_tokens = outputs.sequences[:, input_length:]
                calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_mmse")
                indices = (generated_tokens.cpu()[0] == tokenizer(".Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer("Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer(" Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer(".T")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices) > 0:
                  first_occurrence = indices[0]
                  row= first_occurrence.item()
                  values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
                  decoded_text_mmse = tokenizer.decode(values_before_8)
                  inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+decoded_text_mmse+" Treatment: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
                  outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
                  transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
                  input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
                  generated_tokens = outputs.sequences[:, input_length:]
                  calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_treatment")
                  entry = {
                    "instruction":  instruction_llm,
                    "input":dict_data["input"],
                    "output": "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+decoded_text_mmse+" Treatment: This "+tokenizer.decode(generated_tokens.cpu()[0]),
                    "index":i
                }

        print(entry["output"])
        data_for_json.append(entry)
        with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
           json.dump(data_for_json, jsonfile, indent=4, ensure_ascii=False)


{'ĠNegative': [(5, 0.3799775540828705)], 'ĠIntermediate': [(5, 0.20023411512374878), (16, 0.00018851434288080782)], 'ĠWeak': [(5, 0.05062711238861084)], 'ĠClearly': [(5, 0.03762287274003029)], 'ĠUnknown': [(4, 0.00016461327322758734), (5, 0.08747579902410507), (10, 0.004275469109416008), (16, 0.0002874500351026654), (22, 0.00024558615405112505), (23, 0.001729366136714816)], 'ĠSlow': [(22, 1.2419563972798642e-05), (23, 0.0004237994144205004), (27, 0.4898024797439575)], 'ĠModerate': [(16, 0.00037784597952850163), (23, 0.00011317641474306583), (27, 0.20739541947841644)], 'ĠFast': [(27, 0.028509920462965965)], 'ĠStable': [(27, 0.2313663214445114)], 'ĠVery': [(5, 0.0033654095605015755), (9, 9.892625530483201e-05), (10, 0.0008686142391525209), (15, 2.742817741818726e-06), (16, 0.4920628070831299), (27, 0.00014614070823881775)], 'ĠLow': [(16, 0.3381895124912262), (17, 0.07777095586061478)], 'ĠHigh': [(15, 2.7002940896636574e-06), (16, 0.15727272629737854), (17, 0.0004017774772364646)], 'ĠSupe

In [ ]:
def calculate_probability_new(outputs, generated_tokens,i,tail=""):
  scores = outputs.scores
  from torch.nn.functional import softmax
  top_k = 20  # Define the number of top tokens to consider
  probabilities = [softmax(score, dim=-1) for score in scores]
  generated_tokens = tokenizer.convert_ids_to_tokens(generated_tokens[0])
  import torch
  top_k_tokens_and_probs = []
  for position_probs in probabilities:
    top_k_probs, top_k_indices = torch.topk(position_probs[0], top_k)
    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())
    top_k_tokens_and_probs.append(list(zip(top_k_tokens, top_k_probs.tolist()))) # all top 10 tokens in each generated positions
  expected_tokens=["ĠNegative", "ĠIntermediate", "ĠWeak", "ĠClearly", "ĠUnknown", "ĠSlow", "ĠModerate", "ĠFast", "ĠStable", "ĠVery", "ĠLow", "ĠHigh", "ĠSuper","Ġideal","Ġunlikely"]
  # if abeta_answer=="Unknown" and tau_answer=="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer!="Unknown" and tau_answer=="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating","ĠClearly", "ĠIntermediate", "ĠWeak", "ĠNegative","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer=="Unknown" and tau_answer!="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating", "ĠIntermediate","ĠLow","ĠVery","ĠHigh","ĠSuper","Ġincreasing","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer!="Unknown" and tau_answer!="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating","ĠClearly", "ĠIntermediate", "ĠWeak", "ĠNegative", "ĠLow","ĠHigh","ĠSuper","Ġincreasing","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  expected_token_ids = [tokenizer.convert_tokens_to_ids(token) for token in expected_tokens]

  token_probabilities = {token: [] for token in expected_tokens}

  for idx, token_prob_list in enumerate(top_k_tokens_and_probs):
    for token, prob in token_prob_list:
        if token in expected_tokens:
            token_probabilities[token].append((idx, prob))
  import json
  # Specify the file path
  file_path = f'/content/drive/MyDrive/AD_Jan_3_Zero/POINTER/probability/output_{i}'+tail+'.json'
  print(token_probabilities)
  with open(file_path, 'w') as json_file:
    json.dump(token_probabilities, json_file, indent=4)
  return 0

In [ ]:
## Pointer

In [ ]:
import numpy as np
import bigjson
import json
import torch
import torch.nn.functional as F
import re
decrease_answer=[]
not_answer=[]
decrease_prob=[]
not_prob=[]
json_filepath="/content/drive/MyDrive/AD_Jan_3_Zero/POINTER/pointer_generation.json"

data_for_json=[]
with open('/content/drive/MyDrive/AD_Dec_27/data/pointer_test_Dec27.json', 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
        dict_data = data.to_python()
        values = extract_answers(dict_data["output"], patterns)
        abeta_status=values["Abeta"]
        meta_status=values["Meta tau"]
        mmse_rate=values["MMSE rate"]
        treatment_status=values["Treatment"]
        if abeta_status=="Unknown." and meta_status=="Unknown." and mmse_rate!="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: Unknown. Meta tau: Unknown. MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output": "Abeta: Unknown. Meta tau: Unknown. MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }
            calculate_probability_new(outputs,generated_tokens,i,"other_unknown_mmse")
        if abeta_status!="Unknown." and meta_status=="Unknown." and mmse_rate=="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                   instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens,i,"other_unknown_abeta")
            entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output": "Abeta: This "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }

        if abeta_status!="Unknown." and meta_status=="Unknown." and mmse_rate!="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens =30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens,i,"abeta_known_tau_unknown_mmse_known_abeta")

            indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_abeta = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+ decoded_text_abeta+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_known_tau_unknown_mmse_known_mmse")
              entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output":  "Abeta: This "+ decoded_text_abeta+" MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }

        if abeta_status=="Unknown." and meta_status!="Unknown." :

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: Unknown. Meta tau: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
        #
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True, return_dict_in_generate=True,do_sample=False,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens, i,"abeta_unknown_tau_known_meta_tau")

            indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_new_meta_tau = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: Unknown. Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_unknown_tau_known_mmse")
              entry = {
                    "instruction":  instruction_llm,
                    "input":dict_data["input"],
                    "output": "Abeta: Unknown. Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                    "index":i
                }
        if abeta_status!="Unknown." and meta_status!="Unknown." :

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
        #
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True, return_dict_in_generate=True,do_sample=False,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens, i,"abeta_tau_known_abeta")

            indices = (generated_tokens.cpu()[0] == tokenizer(".Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_new_tau = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+decoded_text_new_tau+" Meta tau: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_meta_tau")
              indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices) > 0:
                first_occurrence = indices[0]
                row= first_occurrence.item()
                values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
                decoded_text_new_meta_tau = tokenizer.decode(values_before_8)
                inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
                outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
                transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
                input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
                generated_tokens = outputs.sequences[:, input_length:]
                calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_mmse")
                indices = (generated_tokens.cpu()[0] == tokenizer(".Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer("Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer(" Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer(".T")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices) > 0:
                  first_occurrence = indices[0]
                  row= first_occurrence.item()
                  values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
                  decoded_text_mmse = tokenizer.decode(values_before_8)
                  inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+decoded_text_mmse+" Treatment: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
                  outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
                  transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
                  input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
                  generated_tokens = outputs.sequences[:, input_length:]
                  calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_treatment")
                  entry = {
                    "instruction":  instruction_llm,
                    "input":dict_data["input"],
                    "output": "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+decoded_text_mmse+" Treatment: This "+tokenizer.decode(generated_tokens.cpu()[0]),
                    "index":i
                }

        print(entry["output"])
        data_for_json.append(entry)
        with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
           json.dump(data_for_json, jsonfile, indent=4, ensure_ascii=False)


In [ ]:
def calculate_probability_new(outputs, generated_tokens,i,tail=""):
  scores = outputs.scores
  from torch.nn.functional import softmax
  top_k = 20  # Define the number of top tokens to consider
  probabilities = [softmax(score, dim=-1) for score in scores]
  generated_tokens = tokenizer.convert_ids_to_tokens(generated_tokens[0])
  import torch
  top_k_tokens_and_probs = []
  for position_probs in probabilities:
    top_k_probs, top_k_indices = torch.topk(position_probs[0], top_k)
    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())
    top_k_tokens_and_probs.append(list(zip(top_k_tokens, top_k_probs.tolist()))) # all top 10 tokens in each generated positions
  expected_tokens=["ĠNegative", "ĠIntermediate", "ĠWeak", "ĠClearly", "ĠUnknown", "ĠSlow", "ĠModerate", "ĠFast", "ĠStable", "ĠVery", "ĠLow", "ĠHigh", "ĠSuper","Ġideal","Ġunlikely"]
  # if abeta_answer=="Unknown" and tau_answer=="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer!="Unknown" and tau_answer=="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating","ĠClearly", "ĠIntermediate", "ĠWeak", "ĠNegative","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer=="Unknown" and tau_answer!="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating", "ĠIntermediate","ĠLow","ĠVery","ĠHigh","ĠSuper","Ġincreasing","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  # if abeta_answer!="Unknown" and tau_answer!="Unknown":
  #   expected_tokens = ["ĠExtremely", "ĠFast", "ĠSlow", "ĠVery","Ġto","Ġnot","ĠModerate","Ġaccumulating","ĠClearly", "ĠIntermediate", "ĠWeak", "ĠNegative", "ĠLow","ĠHigh","ĠSuper","Ġincreasing","Ġextremely", "Ġfast", "Ġslow","Ġvery"]
  expected_token_ids = [tokenizer.convert_tokens_to_ids(token) for token in expected_tokens]

  token_probabilities = {token: [] for token in expected_tokens}

  for idx, token_prob_list in enumerate(top_k_tokens_and_probs):
    for token, prob in token_prob_list:
        if token in expected_tokens:
            token_probabilities[token].append((idx, prob))
  import json
  # Specify the file path
  file_path = f'/content/drive/MyDrive/AD_Jan_3_Zero/A4/probability/output_{i}'+tail+'.json'
  print(token_probabilities)
  with open(file_path, 'w') as json_file:
    json.dump(token_probabilities, json_file, indent=4)
  return 0

In [ ]:
## A4

In [ ]:
import numpy as np
import bigjson
import json
import torch
import torch.nn.functional as F
import re
decrease_answer=[]
not_answer=[]
decrease_prob=[]
not_prob=[]
json_filepath="/content/drive/MyDrive/AD_Jan_3_Zero/A4/A4_generation.json"

data_for_json=[]
with open('/content/drive/MyDrive/AD_Dec_27/data/A4_validation_Dec27.json', 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
        dict_data = data.to_python()
        values = extract_answers(dict_data["output"], patterns)
        abeta_status=values["Abeta"]
        meta_status=values["Meta tau"]
        mmse_rate=values["MMSE rate"]
        treatment_status=values["Treatment"]
        if abeta_status=="Unknown." and meta_status=="Unknown." and mmse_rate!="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: Unknown. Meta tau: Unknown. MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output": "Abeta: Unknown. Meta tau: Unknown. MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }
            calculate_probability_new(outputs,generated_tokens,i,"other_unknown_mmse")
        if abeta_status!="Unknown." and meta_status=="Unknown." and mmse_rate=="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                   instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens,i,"other_unknown_abeta")
            entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output": "Abeta: This "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }

        if abeta_status!="Unknown." and meta_status=="Unknown." and mmse_rate!="Unknown.":

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
            outputs =model.generate(**inputs, max_new_tokens =30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens,i,"abeta_known_tau_unknown_mmse_known_abeta")

            indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_abeta = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+ decoded_text_abeta+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_known_tau_unknown_mmse_known_mmse")
              entry = {
                # "RID":dict_data["RID"],
                # "Visit time":dict_data["Visit time"],
                "instruction":  instruction_llm,
                "input":dict_data["input"],
                "output":  "Abeta: This "+ decoded_text_abeta+" MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                "index":i
            }

        if abeta_status=="Unknown." and meta_status!="Unknown." :

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: Unknown. Meta tau: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
        #
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True, return_dict_in_generate=True,do_sample=False,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens, i,"abeta_unknown_tau_known_meta_tau")

            indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_new_meta_tau = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: Unknown. Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_unknown_tau_known_mmse")
              entry = {
                    "instruction":  instruction_llm,
                    "input":dict_data["input"],
                    "output": "Abeta: Unknown. Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+tokenizer.decode(generated_tokens.cpu()[0]),
                    "index":i
                }
        if abeta_status!="Unknown." and meta_status!="Unknown." :

            inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
        #
            outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True, return_dict_in_generate=True,do_sample=False,temperature=1.0)
            transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            calculate_probability_new(outputs,generated_tokens, i,"abeta_tau_known_abeta")

            indices = (generated_tokens.cpu()[0] == tokenizer(".Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer("Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(" Meta")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices)==0:
              indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
            if len(indices) > 0:
              first_occurrence = indices[0]
              row= first_occurrence.item()
              values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
              decoded_text_new_tau = tokenizer.decode(values_before_8)
              inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+decoded_text_new_tau+" Meta tau: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
              outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
              transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
              input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
              generated_tokens = outputs.sequences[:, input_length:]
              calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_meta_tau")
              indices = (generated_tokens.cpu()[0] == tokenizer(".MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer("MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer(" MMSE")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices)==0:
                indices = (generated_tokens.cpu()[0] == tokenizer(".M")["input_ids"][1]).nonzero(as_tuple=False)
              if len(indices) > 0:
                first_occurrence = indices[0]
                row= first_occurrence.item()
                values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
                decoded_text_new_meta_tau = tokenizer.decode(values_before_8)
                inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                     "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
                outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
                transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
                input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
                generated_tokens = outputs.sequences[:, input_length:]
                calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_mmse")
                indices = (generated_tokens.cpu()[0] == tokenizer(".Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer("Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer(" Treatment")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices)==0:
                  indices = (generated_tokens.cpu()[0] == tokenizer(".T")["input_ids"][1]).nonzero(as_tuple=False)
                if len(indices) > 0:
                  first_occurrence = indices[0]
                  row= first_occurrence.item()
                  values_before_8 = generated_tokens.cpu()[0][0:(row)].tolist()
                  decoded_text_mmse = tokenizer.decode(values_before_8)
                  inputs = tokenizer(
            [
                alpaca_prompt.format(
                    instruction_llm, # instruction
                    dict_data["input"], # input
                    "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+decoded_text_mmse+" Treatment: This ", # output - leave this blank for generation!
                    )
                ], return_tensors = "pt").to("cuda")
                  outputs =model.generate(**inputs, max_new_tokens = 30, use_cache = True,output_scores=True,do_sample=False, return_dict_in_generate=True,temperature=1.0)
                  transition_scores = model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                    )
                  input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
                  generated_tokens = outputs.sequences[:, input_length:]
                  calculate_probability_new(outputs,generated_tokens,i,"abeta_tau_known_treatment")
                  entry = {
                    "instruction":  instruction_llm,
                    "input":dict_data["input"],
                    "output": "Abeta: This "+decoded_text_new_tau+" Meta tau: This "+decoded_text_new_meta_tau+" MMSE rate: The "+decoded_text_mmse+" Treatment: This "+tokenizer.decode(generated_tokens.cpu()[0]),
                    "index":i
                }

        print(entry["output"])
        data_for_json.append(entry)
        with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
           json.dump(data_for_json, jsonfile, indent=4, ensure_ascii=False)


In [ ]:
## Pointer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


import pandas as pd
import numpy as np
import bigjson
import json
import torch
import torch.nn.functional as F
import re
# Define the structured input text
# Set model to evaluation mode
model.eval()
with open('/content/drive/MyDrive/AD_Dec_27/data/pointer_test_Dec27.json', 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
        dict_data = data.to_python()
        input_ids = tokenizer.encode(

                  alpaca_prompt.format(
                      instruction_llm, # instruction
                      dict_data["input"], # input
                      "Abeta", # output - leave this blank for generation!
                      ), return_tensors="pt"
                  )
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        input_ids = input_ids.to(device)
        model.to(device)
        torch.cuda.empty_cache()

        with torch.no_grad():
          outputs = model(input_ids, output_attentions=True)
          attentions = outputs.attentions  # A tuple of attention matrices for each layer
      #   first_token_attention = [layer[:, :, -1, :-1] for layer in attentions]  # Focus on the attention to the start token (dimension: sequence_length-1)

      # # Aggregate attention across heads and layers for simplicity
      #   mean_attention_across_layers = torch.mean(torch.stack(first_token_attention), dim=0)  # (batch_size, num_heads, sequence_length-1)
      #   mean_attention_across_heads = torch.mean(mean_attention_across_layers, dim=1).squeeze().cpu().numpy()  # (sequence_length-1,)

      # # Map attention values back to tokens
      #   tokens = tokenizer.convert_ids_to_tokens(input_ids[0][:-1])  # Exclude the added token when mapping to tokens
      #   attention_by_token = {token: attention for token, attention in zip(tokens, mean_attention_across_heads)}

        first_token_attention = [layer[:, :, -1,  :] for layer in attentions]

        mean_attention_across_layers = torch.sum(torch.stack(first_token_attention), dim=0)  # (batch_size, num_heads, sequence_length)
        mean_attention_across_heads = torch.sum(mean_attention_across_layers, dim=1).squeeze().cpu().numpy()  # (sequence_length,)

          # Map attention values back to tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        attention_by_token = pd.DataFrame({"token": list(tokens), "attention": list (mean_attention_across_heads)})
        attention_by_token.to_csv("/content/drive/MyDrive/AD_attention_Dec_27/POINTER/sample_"+str(i)+".csv")

LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [ ]:
## HABS-HD
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


import pandas as pd
import numpy as np
import bigjson
import json
import torch
import torch.nn.functional as F
import re
# Define the structured input text
# Set model to evaluation mode
model.eval()
with open('/content/drive/MyDrive/AD_Dec_27/data/habs_test_Dec27.json', 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
        dict_data = data.to_python()
        input_ids = tokenizer.encode(

                  alpaca_prompt.format(
                      instruction_llm, # instruction
                      dict_data["input"], # input
                      "Abeta", # output - leave this blank for generation!
                      ), return_tensors="pt"
                  )
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        input_ids = input_ids.to(device)
        model.to(device)
        torch.cuda.empty_cache()

        with torch.no_grad():
          outputs = model(input_ids, output_attentions=True)
          attentions = outputs.attentions  # A tuple of attention matrices for each layer
      #   first_token_attention = [layer[:, :, -1, :-1] for layer in attentions]  # Focus on the attention to the start token (dimension: sequence_length-1)

      # # Aggregate attention across heads and layers for simplicity
      #   mean_attention_across_layers = torch.mean(torch.stack(first_token_attention), dim=0)  # (batch_size, num_heads, sequence_length-1)
      #   mean_attention_across_heads = torch.mean(mean_attention_across_layers, dim=1).squeeze().cpu().numpy()  # (sequence_length-1,)

      # # Map attention values back to tokens
      #   tokens = tokenizer.convert_ids_to_tokens(input_ids[0][:-1])  # Exclude the added token when mapping to tokens
      #   attention_by_token = {token: attention for token, attention in zip(tokens, mean_attention_across_heads)}

        first_token_attention = [layer[:, :, -1,  :] for layer in attentions]

        mean_attention_across_layers = torch.sum(torch.stack(first_token_attention), dim=0)  # (batch_size, num_heads, sequence_length)
        mean_attention_across_heads = torch.sum(mean_attention_across_layers, dim=1).squeeze().cpu().numpy()  # (sequence_length,)

          # Map attention values back to tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        attention_by_token = pd.DataFrame({"token": list(tokens), "attention": list (mean_attention_across_heads)})
        attention_by_token.to_csv("/content/drive/MyDrive/AD_attention_Dec_27/HABS-HD/sample_"+str(i)+".csv")

In [ ]:
## ADNI
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


import pandas as pd
import numpy as np
import bigjson
import json
import torch
import torch.nn.functional as F
import re
# Define the structured input text
# Set model to evaluation mode
model.eval()
with open('/content/drive/MyDrive/AD_Dec_27/data/adni_test_Dec27.json', 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
      if i>=865 and i<=2200 :
        dict_data = data.to_python()
        input_ids = tokenizer.encode(

                  alpaca_prompt.format(
                      instruction_llm, # instruction
                      dict_data["input"], # input
                      "Abeta", # output - leave this blank for generation!
                      ), return_tensors="pt"
                  )
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        input_ids = input_ids.to(device)
        model.to(device)
        torch.cuda.empty_cache()

        with torch.no_grad():
          outputs = model(input_ids, output_attentions=True)
          attentions = outputs.attentions  # A tuple of attention matrices for each layer
      #   first_token_attention = [layer[:, :, -1, :-1] for layer in attentions]  # Focus on the attention to the start token (dimension: sequence_length-1)

      # # Aggregate attention across heads and layers for simplicity
      #   mean_attention_across_layers = torch.mean(torch.stack(first_token_attention), dim=0)  # (batch_size, num_heads, sequence_length-1)
      #   mean_attention_across_heads = torch.mean(mean_attention_across_layers, dim=1).squeeze().cpu().numpy()  # (sequence_length-1,)

      # # Map attention values back to tokens
      #   tokens = tokenizer.convert_ids_to_tokens(input_ids[0][:-1])  # Exclude the added token when mapping to tokens
      #   attention_by_token = {token: attention for token, attention in zip(tokens, mean_attention_across_heads)}

        first_token_attention = [layer[:, :, -1,  :] for layer in attentions]

        mean_attention_across_layers = torch.sum(torch.stack(first_token_attention), dim=0)  # (batch_size, num_heads, sequence_length)
        mean_attention_across_heads = torch.sum(mean_attention_across_layers, dim=1).squeeze().cpu().numpy()  # (sequence_length,)

          # Map attention values back to tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        attention_by_token = pd.DataFrame({"token": list(tokens), "attention": list (mean_attention_across_heads)})
        attention_by_token.to_csv("/content/drive/MyDrive/AD_attention_Dec_27/ADNI/sample_"+str(i)+".csv")

LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


OutOfMemoryError: CUDA out of memory. Tried to allocate 778.00 MiB. GPU 0 has a total capacity of 22.17 GiB of which 510.88 MiB is free. Process 203659 has 21.66 GiB memory in use. Of the allocated memory 20.30 GiB is allocated by PyTorch, and 1.13 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
i

872